In [2]:
import pandas as pd
import numpy as np

from pulp import *

In [11]:
data_file = "data/fl_2000_2"
data = pd.read_csv(data_file, sep=" ", names=["a", "b", "c", "d"], dtype={"a":float, "b":float, "c":float, "d":float})

In [12]:
data.head()

,a,b,c,d
0,2000.0000,2000.0,NaN,NaN
1,8870.0003,4406.0,39614.6550,87073.9800
2,9516.2710,4998.0,97630.5325,132.2800
3,7038.8687,3683.0,73246.5375,27828.4450
4,10199.1305,4621.0,54702.5450,4158.2225


In [13]:
n = int(data["a"][0])
m = int(data["b"][0])
print("n=%s" % n)
print("m=%s" % m)

n=2000
m=2000


In [14]:
facilities = data[1:(n+1)].rename({"a":"cost", "b":"capacity", "c":"x", "d":"y"}, axis="columns").reset_index(drop=True)
facilities.head()

,cost,capacity,x,y
0,8870.0003,4406.0,39614.6550,87073.9800
1,9516.2710,4998.0,97630.5325,132.2800
2,7038.8687,3683.0,73246.5375,27828.4450
3,10199.1305,4621.0,54702.5450,4158.2225
4,8931.4245,6243.0,180.0075,76980.6025


In [15]:
customers = data[(n+1):].drop("d", axis=1).rename({"a":"demand", "b":"x", "c":"y"}, axis="columns").reset_index(drop=True)
customers.head()

,demand,x,y
0,1101.0,41708.9700,91744.6825
1,1330.0,63151.5125,5834.0400
2,1297.0,83082.8650,68100.6875
3,1533.0,12593.5075,46299.3700
4,1053.0,71175.1700,61375.5250


We want to minimise the total cost of satifysing customer demand from a series of facilities (warehouses)

#### Description of rules and constraints

- Each customer must be served by exactly one facility
- The total demand of the customers cannot exceed the capacity of the facility

#### Notation and parameters

- $F$ is the set of facilities
- $C$ is the set of customers
- $f_i , \forall i\in F$ is the capacity of each facility
- $c_j , \forall j\in C$ is the demand of each customer
- $d_{ij} , \forall i \in F, j \in C$ is the euclidian distance between facility $j$ and customer $i$
- $s_i , \forall i\in F$ is the fixed cost for operating facility $i$

Note: In this example the euclidian distance between facility $j$ and customer $i$ is considered the transport cost so we can just add them to the fixed costs

#### Decision variables

$ x_{ij} \forall i\in F, \forall j\in C $ is whether customer $j$ is served by facility $i$

$x_{ij} \in \{0,1\}$

$p_i \forall i\in F$ is whether or not facility $i$ is used

$p_{i} \in \{0,1\}$

#### Objective function (minimize)

$$ \min (\sum_{i\in F}\sum_{j\in C}d_{ij}x_{ij} + \sum_{i \in F}p_i s_i) $$

#### Constraints

$$ \sum_{i \in F} x_{ij} = 1, \forall j \in C\; (1)$$

$$ \sum_{j \in C} x_{ij}c_{j} \le f_i, \forall i \in F\; (2)$$

$$ \sum_{j \in C} x_{ij} \le \ bigM p_i, \forall i \in F\; (3)$$

Alternative (stronger linear relaxation)

$$ \sum_{i \in F} x_{ij} = 1, \forall j \in C\; (1)$$

$$ \sum_{j \in C} x_{ij}c_{j} \le f_i, \forall i \in F\; (2)$$

$$ x_{ij} \le p_i, \forall i \in F\,\; \forall j \in C\; (3)$$


In [16]:
def euclidean_distance(x_1, x_2, y_1, y_2):
    return np.sqrt( np.square(x_1 - x_2)  + np.square(y_1 - y_2) )

F = list(range(0, n))
C = list(range(0, m))
f = np.array(facilities.capacity)
c = np.array(customers.demand)
s = np.array(facilities.cost)
d = np.zeros([n, m])

all_pairs = [(i, j) for i in F for j in C]
facility_locations = np.array(facilities[["x", "y"]])
customer_locations = np.array(customers[["x", "y"]])

for pair in all_pairs:
    i = pair[0]
    j = pair[1]
    d[i,j] = euclidean_distance(facility_locations[i,0], customer_locations[j,0], facility_locations[i,1], customer_locations[j,1])
    


In [ ]:
### Create Problem/ Solver
prob = LpProblem("The facility location problem", LpMinimize)

### Create decision variables
xij = {}
for i in F:
    for j in C:
        xij[i,j] = LpVariable("x_%s_%s" % (i,j), 0, 1, LpBinary)
        
pi = {}
for i in F:
    pi[i] = LpVariable("p_%s" % (i), 0, 1, LpBinary)
    
## Set the objective function 
prob += lpSum([d[(i,j)]*xij[i,j] for i in F for j in C]) + lpSum([s[i] * pi[i] for i in F])

In [253]:
## Set constraint 1
# Using great than or equal is significantly faster
for j in C:
    prob += lpSum([xij[i,j] for i in F]) == 1,"%s must be assigned to at least one facility" % (j)

In [254]:
## Set constraint 2
for i in F:
    prob += lpSum([xij[i,j] * c[j] for j in C]) <= f[i],"%s demand must not exceed capacity" % (i)

In [255]:
## Set constraint 3
#bigM=m
#for i in F:
#    prob += lpSum([xij[i,j] for j in C]) <= bigM * pi[i],"%s bigM constraint" % (i)

for i in F:
    for j in C:
        prob += xij[i,j] <= pi[i]

In [256]:
## Check model
#prob

In [273]:
import time

start = time.time()
## Solve model
prob.solve(solvers.PULP_CBC_CMD(maxSeconds=1200, threads=3,fracGap=0.0001))
end = time.time()
print(pulp.LpStatus[prob.status], "solution is: ", float(end - start), "sec")

Optimal solution is:  4.35257887840271 sec


In [258]:
## Print variables with value greater than 0 
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

# Print The optimal objective function value
print("Total Cost = ", pulp.value(prob.objective))

p_68 = 1.0
x_68_0 = 1.0
x_68_1 = 1.0
x_68_10 = 1.0
x_68_100 = 1.0
x_68_101 = 1.0
x_68_102 = 1.0
x_68_103 = 1.0
x_68_104 = 1.0
x_68_105 = 1.0
x_68_106 = 1.0
x_68_107 = 1.0
x_68_108 = 1.0
x_68_109 = 1.0
x_68_11 = 1.0
x_68_110 = 1.0
x_68_111 = 1.0
x_68_112 = 1.0
x_68_113 = 1.0
x_68_114 = 1.0
x_68_115 = 1.0
x_68_116 = 1.0
x_68_117 = 1.0
x_68_118 = 1.0
x_68_119 = 1.0
x_68_12 = 1.0
x_68_120 = 1.0
x_68_121 = 1.0
x_68_122 = 1.0
x_68_123 = 1.0
x_68_124 = 1.0
x_68_125 = 1.0
x_68_126 = 1.0
x_68_127 = 1.0
x_68_128 = 1.0
x_68_129 = 1.0
x_68_13 = 1.0
x_68_130 = 1.0
x_68_131 = 1.0
x_68_132 = 1.0
x_68_133 = 1.0
x_68_134 = 1.0
x_68_135 = 1.0
x_68_136 = 1.0
x_68_137 = 1.0
x_68_138 = 1.0
x_68_139 = 1.0
x_68_14 = 1.0
x_68_140 = 1.0
x_68_141 = 1.0
x_68_142 = 1.0
x_68_143 = 1.0
x_68_144 = 1.0
x_68_145 = 1.0
x_68_146 = 1.0
x_68_147 = 1.0
x_68_148 = 1.0
x_68_149 = 1.0
x_68_15 = 1.0
x_68_150 = 1.0
x_68_151 = 1.0
x_68_152 = 1.0
x_68_153 = 1.0
x_68_154 = 1.0
x_68_155 = 1.0
x_68_156 = 1.0
x_68_157 = 1.0
x_68_158 

In [259]:
## Print variables with value greater than 0 
customer_assignments = {}
for v in prob.variables():
    if v.varValue > 0:
        var_split = v.name.split("_")
        if (var_split[0] == "x"):
            customer_assignments[var_split[2]] = var_split[1]

settings = [customer_assignments[str(i)] for i in range(0,m)]

In [274]:
# prepare the solution in the specified output format
is_opt = str(1) if pulp.LpStatus[prob.status] == "Optimal" else str(0)

output_data = str(pulp.value(prob.objective)) + ' ' + is_opt + '\n'
output_data += ' '.join(map(str, settings))
print(output_data)

3807.322699189734 1
68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68 68


In [245]:
m

200

In [261]:
pulp.pulpTestAll()

	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver <class 'pulp.solvers.PULP_CBC_CMD'> passed.
Solver <class 'pulp.solvers.CPLEX_DLL'> un

In [9]:
np.array(facilities[["x", "y"]])

array([[34.824276, 34.824276],
       [34.905834, 38.077609],
       [43.586961, 28.2197  ],
       [40.395677, 33.445805],
       [35.072047, 35.180733],
       [37.41294 , 30.268366],
       [36.244985, 34.411039],
       [42.747881, 29.513173],
       [35.331494, 34.542876],
       [46.126615, 25.24013 ],
       [36.378112, 36.506686],
       [40.723124, 31.774915],
       [38.554008, 34.329116],
       [39.436221, 31.616298],
       [39.738381, 28.215364],
       [39.082726, 28.637864],
       [44.616617, 29.116289],
       [38.824557, 31.421501],
       [39.924155, 29.423705],
       [48.008579, 26.250543],
       [44.084714, 26.487049],
       [42.724702, 30.53449 ],
       [39.494771, 30.008531],
       [39.5547  , 31.169539],
       [45.386696, 27.886432],
       [37.036494, 39.870722],
       [35.321085, 37.32953 ],
       [43.772532, 27.080263],
       [37.552833, 31.59753 ],
       [47.902814, 23.803999],
       [44.41772 , 28.187908],
       [37.485153, 32.17322 ],
       [